In [2]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

FILE_AGRUPACIONES = 'data/sku_unification_v6.xlsx'
CONVERSIONES_UNIDADES = 'data/conversiones_unidades.csv'
OUTPUT_FILE = 'data/ahorro_por_cd.csv'

ventas_2018 = pd.read_csv('data/ventas_2018.csv', encoding = "ISO-8859-1")
ventas_2019 = pd.read_csv('data/ventas_2019.csv', encoding = "ISO-8859-1")
ventas_2020 = pd.read_csv('data/ventas_2020.csv', encoding = "ISO-8859-1")

ventas_2018['PERIODO'] = ventas_2018.apply(lambda x: date(2018, int(str(x.PERIODO)[4:]), 1), axis=1)
ventas_2018 = ventas_2018.drop(columns=['DESCR_ZONENT'], axis=1)

ventas_2019['PERIODO'] = ventas_2019.apply(lambda x: date(2019, int(str(x.PERIODO)[4:]), 1), axis=1)
ventas_2019 = ventas_2019.drop(columns=['DESCR_ZONENT'], axis=1)

ventas_2020['PERIODO'] = ventas_2020.apply(lambda x: date(2020, int(str(x.PERIODO)[4:]), 1), axis=1)
ventas_2020 = ventas_2020.drop(columns=['DESCR_ZONENT'], axis=1)

ventas_2018 = ventas_2018.astype({'PERIODO': 'datetime64[ns]'})

ventas_2019 = ventas_2019.astype({'PERIODO': 'datetime64[ns]'})

ventas_2020 = ventas_2020.astype({'PERIODO': 'datetime64[ns]'})

with open('data/skus_filter.json', 'r') as file:
    skus_filter = json.load(file)

ventas_finales = pd.concat([ventas_2018, ventas_2019, ventas_2020]).sort_values('PERIODO')

ventas_finales = ventas_finales[~ventas_finales['ID_SKU_VENTA'].isin(skus_filter)]
ventas_finales = ventas_finales.drop(columns=['DESCR_PROD'], axis=1)

confianza = 1.65

hubs_dict = {'ARICA': 'IQUIQUE',
 'IQUIQUE': 'IQUIQUE',
 'ANTOFAGASTA': 'ANTOFAGASTA',
 'COPIAPO': 'COPIAPO',
 'COQUIMBO': 'COQUIMBO',
 'OVALLE': 'COQUIMBO',
 'ILLAPEL': 'CURAUMA',
 'LLAY LLAY': 'CURAUMA',
 'CURAUMA': 'CURAUMA',
 'CERVECERA': 'SANTIAGO SUR',
 'MODELO': 'SANTIAGO SUR',
 'SANTIAGO SUR': 'SANTIAGO SUR',
 'RANCAGUA': 'SANTIAGO SUR',
 'TALCA': 'TALCA',
 'CHILLAN': 'TALCAHUANO',
 'TALCAHUANO': 'TALCAHUANO',
 'LOS ANGELES': 'TALCAHUANO',
 'TEMUCO': 'TEMUCO',
 'VALDIVIA': 'VALDIVIA',
 'OSORNO': 'VALDIVIA',
 'PUERTO MONTT': 'PUERTO MONTT',
 'CASTRO': 'PUERTO MONTT',
 'COYHAIQUE': 'COYHAIQUE',
 'CALAMA': 'ANTOFAGASTA'}

ventas_finales = ventas_finales[ventas_finales['DESCR_CENDIST'].isin(list(hubs_dict.keys()))]
ventas_finales = ventas_finales.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'PERIODO']).sum().reset_index()

ventas_finales.to_csv('data/ventas_finales.csv')

print('ARCHIVO LISTO')

means_stddev_dict = {}
ids = ventas_finales['ID_SKU_VENTA'].drop_duplicates()
cds = ventas_finales['DESCR_CENDIST'].drop_duplicates()

for id_ in ids:
    
    data_id = ventas_finales[ventas_finales['ID_SKU_VENTA'] == id_]
    
    for cd in cds:
        
        if pd.isnull(cd):
            continue
        
        data_id_cd = data_id[data_id['DESCR_CENDIST'] == cd]
        
        #data_id_cd_semana = data_id_cd
        #data_id_cd_semana['SUM(PREVENTA)'] = data_id_cd_semana.apply(lambda x: math.ceil(x['SUM(PREVENTA)'] / 4), axis=1)
        ventas = data_id_cd['SUM(PREVENTA)'].to_numpy()
        
        largo = len(ventas)
        
        if largo == 32:
            mean = ventas.mean() / 4
            std = ventas.std() / 4
        
        else:
            excedente = 32 - largo
            for i in range(excedente):
                ventas = np.append(ventas, [0])
            mean = ventas.mean() / 4
            std = ventas.std() / 4  
        
        if pd.isnull(mean):
            means_stddev_dict[(id_, cd)] = (0, 0)
            continue

        means_stddev_dict[(id_, cd)] = (mean, std)
        
print(means_stddev_dict)      
s_stock_dict = {}

for key in means_stddev_dict:
    
    s_stock_dict[key] = means_stddev_dict[key][0] + (confianza * means_stddev_dict[key][1])
    
ventas_finales_new = ventas_finales.copy()

sheets_dict = pd.read_excel(FILE_AGRUPACIONES, sheet_name=None)

master = pd.DataFrame()
for name in sheets_dict:
    
    df = sheets_dict[name]
    master = pd.concat([master, df])
    
dict_sku_cd = {}
master = master.rename(columns={'SKU': 'ID_SKU_VENTA', 'CD': 'DESCR_CENDIST'})

for index, row in master.iterrows():
    
    if row['CANTIDAD HUB 3R'] != 0:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 1
        
    else:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 0
        
ventas_finales_new['DESCR_CENDIST'] = ventas_finales_new.apply(lambda x: x['DESCR_CENDIST'] if (x['ID_SKU_VENTA'], x['DESCR_CENDIST']) in dict_sku_cd and dict_sku_cd[(x['ID_SKU_VENTA'], x['DESCR_CENDIST'])] == 0 else hubs_dict[x['DESCR_CENDIST']], axis=1)
ventas_finales_new = ventas_finales_new.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'PERIODO']).sum().reset_index()

means_stddev_dict_optimized = {}
ids = ventas_finales_new['ID_SKU_VENTA'].drop_duplicates()
cds = ventas_finales_new['DESCR_CENDIST'].drop_duplicates()


for id_ in ids:
    
    data_id = ventas_finales_new[ventas_finales_new['ID_SKU_VENTA'] == id_]
    
    for cd in cds:
        
        if pd.isnull(cd):
            continue
        
        data_id_cd = data_id[data_id['DESCR_CENDIST'] == cd]
        
        #data_id_cd_semana = data_id_cd
        #data_id_cd_semana['SUM(PREVENTA)'] = data_id_cd_semana.apply(lambda x: math.ceil(x['SUM(PREVENTA)'] / 4), axis=1)
        
        ### MODIFICAR MEDIAS Y STDDEV: MODIFICAR DATAFRAME Y DPS AGRUPAR CHAO DIEGO FUMADOOOO
        ventas = data_id_cd['SUM(PREVENTA)'].to_numpy()
        
        largo = len(ventas)
        
        if largo == 32:
            mean = ventas.mean() / 4
            std = ventas.std() / 4
        
        else:
            excedente = 32 - largo
            for i in range(excedente):
                ventas = np.append(ventas, [0])
            mean = ventas.mean() / 4
            std = ventas.std() / 4
        
        if pd.isnull(mean):
            means_stddev_dict_optimized[(id_, cd)] = (0, 0)
            continue

        means_stddev_dict_optimized[(id_, cd)] = (mean, std)
        
s_stock_dict_optimized = {}

for key in means_stddev_dict_optimized:
    
    s_stock_dict_optimized[key] = means_stddev_dict_optimized[key][0] + (confianza * means_stddev_dict_optimized[key][1])
    
diff_dict = {}

for key in s_stock_dict:
    
    if key in s_stock_dict_optimized:
        
        diff_dict[key] = s_stock_dict_optimized[key] - s_stock_dict[key]
        
    else:
        
        diff_dict[key] = - s_stock_dict[key]
        
conversion = pd.read_csv('data/conversiones_unidades.csv', encoding = "ISO-8859-1")[['ID_SKU_VENTA', 'CAJAS_POR_PALLET']]

sku_pallets = {}

for index, row in conversion.iterrows():
    
    sku_pallets[row['ID_SKU_VENTA']] = row['CAJAS_POR_PALLET']    
        
inventory_save_dict_cd = {}
    
for key in diff_dict:
        
    
    if key[1] in inventory_save_dict_cd:
            inventory_save_dict_cd[key[1]] += diff_dict[key] / sku_pallets[key[0]]
                
    else:
            inventory_save_dict_cd[key[1]] = diff_dict[key] / sku_pallets[key[0]]
            
data_dict = {'CD': list(inventory_save_dict_cd.keys()), 'AHORRO_PALLETS_INVENTARIO': list(inventory_save_dict_cd.values())}

data = pd.DataFrame.from_dict(data_dict)
data.to_csv(OUTPUT_FILE)

ARCHIVO LISTO
{(515, 'ANTOFAGASTA'): (44.634765625, 7.770042590560934), (515, 'ARICA'): (4.2265625, 1.7347092864782128), (515, 'CALAMA'): (23.3203125, 8.811000398782408), (515, 'CASTRO'): (13.875, 8.575947469521953), (515, 'CERVECERA'): (650.3671875, 247.04143902433626), (515, 'CHILLAN'): (22.9375, 10.341814395936527), (515, 'COPIAPO'): (24.421875, 6.64984653840786), (515, 'COQUIMBO'): (77.2578125, 25.375806974160323), (515, 'COYHAIQUE'): (6.90625, 2.527775393008643), (515, 'ILLAPEL'): (10.15625, 5.825171000708906), (515, 'IQUIQUE'): (7.8515625, 4.301098776312136), (515, 'LLAY LLAY'): (1.6953125, 1.0784363228043417), (515, 'LOS ANGELES'): (58.578125, 22.618562404237256), (515, 'OSORNO'): (25.3515625, 9.428360511435365), (515, 'OVALLE'): (13.3125, 7.3529862130973695), (515, 'PUERTO MONTT'): (43.1484375, 11.140063428167442), (515, 'SANTIAGO SUR'): (219.390625, 74.59400365216614), (515, 'TALCA'): (115.4921875, 38.49033382993507), (515, 'TALCAHUANO'): (166.1171875, 51.835564241308724), (51

KeyboardInterrupt: 